# Preprocessing

## Overview of Data

In [1]:
# Import pandas
import pandas as pd

In [2]:
# Read CSV file
df = pd.read_csv('data/original_dataset.csv')

# Preview df
df.head()

,Student_ID,First_Name,Last_Name,Email,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,...,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,S1000,Omar,Williams,student0@university.com,Female,22,Engineering,52.29,55.03,57.82,...,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,S1001,Maria,Brown,student1@university.com,Male,18,Engineering,97.27,97.23,45.80,...,55.65,50.64,A,19.0,No,Yes,NaN,Medium,4,9.0
2,S1002,Ahmed,Jones,student2@university.com,Male,24,Business,57.19,67.05,93.68,...,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,S1003,Omar,Williams,student3@university.com,Female,24,Mathematics,95.15,47.79,80.63,...,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,S1004,John,Smith,student4@university.com,Female,23,CS,54.18,46.59,78.89,...,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1


In [3]:
# Review dataframe columns, datatypes, null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Student_ID                  5000 non-null   object 
 1   First_Name                  5000 non-null   object 
 2   Last_Name                   5000 non-null   object 
 3   Email                       5000 non-null   object 
 4   Gender                      5000 non-null   object 
 5   Age                         5000 non-null   int64  
 6   Department                  5000 non-null   object 
 7   Attendance (%)              4484 non-null   float64
 8   Midterm_Score               5000 non-null   float64
 9   Final_Score                 5000 non-null   float64
 10  Assignments_Avg             4483 non-null   float64
 11  Quizzes_Avg                 5000 non-null   float64
 12  Participation_Score         5000 non-null   float64
 13  Projects_Score              5000 

In [4]:
# Number of null values in DataFrame columns
df.isnull().sum()

Student_ID                       0
First_Name                       0
Last_Name                        0
Email                            0
Gender                           0
Age                              0
Department                       0
Attendance (%)                 516
Midterm_Score                    0
Final_Score                      0
Assignments_Avg                517
Quizzes_Avg                      0
Participation_Score              0
Projects_Score                   0
Total_Score                      0
Grade                            0
Study_Hours_per_Week             0
Extracurricular_Activities       0
Internet_Access_at_Home          0
Parent_Education_Level        1794
Family_Income_Level              0
Stress_Level (1-10)              0
Sleep_Hours_per_Night            0
dtype: int64

In [5]:
# Use Shapiro-Wilk from scipy to test for normality in the numerical columns with null values. Omit NaN values.
from scipy import stats

# Test on Assignments_Avg dataseries
sh_test1 = stats.shapiro(df['Assignments_Avg'], nan_policy= 'omit')

# Test on Attendance (%) dataseries
sh_test2 = stats.shapiro(df['Attendance (%)'], nan_policy= 'omit')

# Interpret and print results
alpha = 0.05

if sh_test1.pvalue > alpha:
    print("Assignments_Avg looks normally distributed")
else: print("Assignments_Avg does not look normally distributed")

if sh_test2.pvalue > alpha:
    print("Attendance (%) looks normally distributed")
else: print("Attendance (%) does not look normally distributed")

Assignments_Avg does not look normally distributed
Attendance (%) does not look normally distributed


In [6]:
# Check value_counts of Parent_Education_Level, which has 1794 null values
df['Parent_Education_Level'].value_counts()

Parent_Education_Level
PhD            820
Bachelor's     810
High School    796
Master's       780
Name: count, dtype: int64

### Strategy regarding null values
Since data is not normally distributed in both of these numeric columns, using the mean to impute data isn't a good idea. Using the median may introduce bias. Therefore, we will use `KNNImputer` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html)) to fill missing numeric values using k-nearest neighbors algorithm.

With regard to categorical variable `Parent_Education_Level` which is 35% null values, we will encode the column with the following values:
* 0 - `High School`
* 1 - `Bachelor's`
* 2 - `Master's`
* 3 - `PhD`

This numeric column will also be included in the `KNNImputer` strategy outlined above.

In ReadME, talk about other strategies considered for null values in `Parent_Education_Level`:

* Impute the mode - inappropriate for machine learning, especially since the mode in this case is `PhD`
* Create new class `Not Provided`, but this undermine an important relationship between the feature and the variable.
* Dropping the column all together. 35% is a lot of missing data. This option may be considered while we train/test/evaluate the model.

## Data Imputation

### Encoding

In [7]:
# Map the Parent_Education_Level data series, assign 0-4 value. Add as column to df. NaN values will remain.
df['Parent_Education_Level_map'] = df['Parent_Education_Level'].map({
    "High School":0,
    "Bachelor's": 1,
    "Master's": 2,
    "PhD": 3})

In [8]:
# Encode Grade (target variable) using `LabelEncoder`
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
encoder = LabelEncoder()

# Use LabelEncoder on target variable. 0 = A, 1 = B, 2 = C, 3 = D, 4 = F. Add as a column in grades_df
df['Encoded_Grade'] = encoder.fit_transform(df['Grade'])

In [9]:
# Drop `Parent_Education_Level`, `Grade` columns and ID/email/name columns
df = df[[x for x in df.columns.tolist() if x not in ('Student_ID','First_Name','Last_Name','Email','Grade','Parent_Education_Level')]]

In [10]:
# Generate categorical variable lists for encoding
categorical = df.dtypes[df.dtypes == 'object'].index.tolist()

In [11]:
# Check number of unique values in categorical variables
df[categorical].nunique()

Gender                        2
Department                    4
Extracurricular_Activities    2
Internet_Access_at_Home       2
Family_Income_Level           3
dtype: int64

In [ ]:
# Encode categorical variables using OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance to encode remaining categorical variables
enc = OneHotEncoder(sparse_output= False)

# Fit and transform OneHotEncoder using categorical variables
encode_df = pd.DataFrame(enc.fit_transform(df[categorical]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(categorical)

In [ ]:
# Use pd.concat to add columns with encoded values
df2 = pd.concat([df,encode_df], axis=1)

# Drop object columns
df2 = df2.select_dtypes(exclude=['object'])

In [ ]:
# Review dataframe before imputation
df2.info()

### Imputing using `KNNImputer`

In [ ]:
from sklearn.impute import KNNImputer

# Initiate KNN Imputer setting nearest neighbors parameter to 10
imputer = KNNImputer(n_neighbors=10)

# Run imputer on DataFrame for missing values, then store results into a new DataFrame
df_imputed = pd.DataFrame(imputer.fit_transform(df2), columns=[x for x in df2.columns])

In [ ]:
# Rename Parent_Education_Level_map
df_imputed = df_imputed.rename(columns={'Parent_Education_Level_map':'Parent_Education_Level'})

# Round values in Parent_Education_Level to whole number
df_imputed['Parent_Education_Level'] = df_imputed['Parent_Education_Level'].round()

In [ ]:
# Review to confirm that there are no null values
df_imputed.info()

In [ ]:
# Save df_imputed to csv for modeling
df_imputed.to_csv('data/preprocessed_grades_data.csv',index=False)